In [2]:
print("manish")

manish


In [35]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\manis\anaconda3\envs\mchatbot\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [36]:
PINECONE_API_KEY="pcsk_7KRq2a_DBuYTodKjiFoChiAHCQAU5WKjVPs6s1pWfY9UyagocMdmv1yVitqMWcRXGmKy5W"
PINECONE_API_KEY="https://negi-o6bdmu5.svc.aped-4627-b74a.pinecone.io"

In [37]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents